# Installing packages:
  
pip install numpy

pip install python-opencv 

In [7]:
import numpy as np
import cv2
import math
import datetime


def relativize(m):
    m[m >= 0] = 1+np.log(1+m[m >= 0])
    m[m < 0] = np.exp(m[m < 0])
    return m


class VideoFilter:
    def __init__(self, file=0):
        self.rate = 0.1       # Update avg with this amount of the new value
        self.power = 1        # Strength of the effect, 0 = None, 1 = ok.
        self.reduce = 1       # Resolution reduction (1, 2 or 4):
        self.max_err = 32     # 10=high sensitivity, 32=noise-free output 
        self.mode = 0         # different ways of showing the results
        self.color_mode = 0   # 0=RGB, 1=grayscxale, 2=HSV 
        self.save_vid = False # Save the resulting video to file, file name is automatic.
        # Open the capture device:
        self.open_video(file)
        self.time1 = datetime.datetime.now()
        self.fps = 0
        
    def open_video(self, file=0):
        # Open webcam device:
        self.cap = cv2.VideoCapture(file)
        # Redefine size of input images:      
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH,  self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)  // self.reduce)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT) // self.reduce)
        # Read the resulting shape (not always accepted by source)
        self.width  = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        # Initialize averages:
        self.ini_memory()

    def ini_memory(self):
        self.get_frame()
        self.avg = np.ones(self.frame.shape, dtype=float)*128
        self.err = np.ones(self.frame.shape, dtype=float)*128
        # Initialize output images:
        self.output = cv2.copyMakeBorder(self.frame, 0, 0, 0, 0, cv2.BORDER_REPLICATE)  # BGR
        self.triple = cv2.copyMakeBorder(self.frame0, 0, 0, 0, self.width, cv2.BORDER_REPLICATE)
        self.last_time = datetime.datetime.now()

    def get_frame(self):
        ret, self.frame0 = self.cap.read()
        if ret:
            if (self.color_mode == 0):
                self.frame = self.frame0
            elif (self.color_mode == 1):
                self.frame = cv2.cvtColor(self.frame0, cv2.COLOR_BGR2GRAY)
            elif (self.color_mode == 2):
                self.frame = cv2.cvtColor(self.frame0, cv2.COLOR_BGR2HSV)
        return ret
       
    def cycle_mode(self):
        self.mode = divmod(self.mode + 1, 3)[1]       

    def cycle_color_mode(self):
        self.color_mode = divmod(self.color_mode + 1, 3)[1]       
        self.ini_memory()
    
    def save_video(self, file_name):
        self.save_vid = True
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.out = cv2.VideoWriter('output_RT.avi', fourcc, 1.0, (self.triple.shape[0], self.triple.shape[1]), True)
        
    def end_work(self):
        self.cap.release()
        cv2.destroyAllWindows()
        if self.save_vid:
            self.out.release        
        
    def process_frame(self):
        if (self.mode == 0):
            # neutral gray frame gets darker or lighter with movement:
            self.coef = relativize((self.frame - self.avg) / self.max_err / 2)
            self.coef = np.power(self.coef, self.power)
            if (self.color_mode == 2):
                self.output[:,:,0] = self.frame[:,:,0]
                self.output[:,:,1] = self.frame[:,:,1]
                self.output[:,:,2] = np.clip(126 * self.coef[:,:,2], 0, 255)
            else:
                self.output = np.clip(126 * self.coef, 0, 255)           
        if (self.mode == 1):
            # black frame gets lighter with movement:
            self.coef = relativize(np.abs(self.frame - self.avg) / self.max_err)   
            self.coef = np.power(self.coef, self.power) - 1
            if (self.color_mode == 2):
                self.output[:,:,0] = self.frame[:,:,0]
                self.output[:,:,1] = self.frame[:,:,1]
                self.output[:,:,2] = np.clip(255 * self.coef[:,:,2], 0, 255)
            else:
                self.output = np.clip(255 * self.coef, 0, 255)
        if (self.mode == 2):
            # white frame get darker with movement:
            self.coef = relativize(np.abs(self.frame - self.avg) / self.max_err / 2)   
            self.coef = np.power(self.coef, self.power) - 1
            if (self.color_mode == 2):
                self.output[:,:,0] = self.frame[:,:,0]
                self.output[:,:,1] = self.frame[:,:,1]
                self.output[:,:,2] = np.clip(255 * (1-self.coef[:,:,2]) + self.frame[:,:,2] * self.coef[:,:,2], 0, 255)
            else:
                self.output = np.clip(255 * (1-self.coef) + self.frame * self.coef, 0, 255)
        
        # Convert output to BGR mode:       
        if (self.color_mode == 1):
            self.output = np.uint8(self.output)
            self.output = cv2.cvtColor(self.output, cv2.COLOR_GRAY2BGR)
        elif (self.color_mode == 2):
            self.output = np.uint8(self.output)
            self.output = cv2.cvtColor(self.output, cv2.COLOR_HSV2BGR)

    def update_avg(self):
        # Update averaged image:
        self.avg = (1-self.rate) * self.avg + self.rate * self.frame        
        
    def update_output(self):
        self.triple[0:self.frame.shape[0], 0:self.frame.shape[1]] = self.frame0
        self.triple[0:self.frame.shape[0], self.frame.shape[1]:self.frame.shape[1]*2] = self.output   
        # Show final composite image:
        cv2.imshow('Relativize video filter', self.triple)    
        # write the frame to video:
        if self.save_vid:
            self.out.write(self.triple)
        # update fps:
        time2 = datetime.datetime.now()
        elapsedTime = time2 - self.time1
        self.fps = 1 / elapsedTime.total_seconds()
        self.time1 = time2
        
    def step(self):
        if self.get_frame():
            self.process_frame()
            self.update_avg()
            self.update_output()
            return True
        else:
            return False
            

In [8]:
my_filter = VideoFilter() # webcam 
# my_filter = VideoFilter('Despierta tu diosa.mp4') 
# my_filter.save_video('output_RT.avi')

while(True):  
    ret = my_filter.step()   
    if ret:
        print('mode:', my_filter.mode, 'fps: ', round(my_filter.fps), end='                 \r')
    # Press q, m or c:    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('m'):
        my_filter.cycle_mode()
    if key == ord('c'):
        my_filter.cycle_color_mode()
    if (key == ord('q')) or (not ret):
        break

my_filter.end_work()
